In [5]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import pandas as pd
import folium
print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = '51BZ0GO2PVOKVHT11Q2LLYYGIUCS1X1IBAB2LAGCA5CUQF41' # your Foursquare ID
CLIENT_SECRET = 'VVB5HYNRGCTI3LMLRTMXHA5BWKWX2RFAMBT52R34S1CN12UH' # your Foursquare Secret
ACCESS_TOKEN = 'H0LX0BFVUG2RPI2SABT3Q2XZREYUKEQS3KAZKCHAYGNVQS3B' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 51BZ0GO2PVOKVHT11Q2LLYYGIUCS1X1IBAB2LAGCA5CUQF41
CLIENT_SECRET:VVB5HYNRGCTI3LMLRTMXHA5BWKWX2RFAMBT52R34S1CN12UH


In [19]:
address = "Rogers centre, Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rogers centre are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rogers centre are 43.6416641, -79.38919882366382.


In [20]:
search_query = 'Coffee'
radius = 2000
LIMIT = 50

In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=51BZ0GO2PVOKVHT11Q2LLYYGIUCS1X1IBAB2LAGCA5CUQF41&client_secret=VVB5HYNRGCTI3LMLRTMXHA5BWKWX2RFAMBT52R34S1CN12UH&ll=43.6416641,-79.38919882366382&oauth_token=H0LX0BFVUG2RPI2SABT3Q2XZREYUKEQS3KAZKCHAYGNVQS3B&v=20180605&query=Coffee&radius=2000&limit=50'

In [22]:
results = requests.get(url).json()

In [23]:
venues = results['response']['venues']

df = pd.json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,59f784dd28122f14f9d5d63d,HotBlack Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612105369,False,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",969,M5V 1Z4,CA,Entertainment District,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",463001529
1,4b05a67cf964a52034e022e3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612105369,False,225 Front Street E,Metro Toronto Convention Centre,43.640346,-79.389881,"[{'label': 'display', 'lat': 43.640346, 'lng':...",156,NaN,CA,NaN,Toronto,ON,Canada,[225 Front Street E (Metro Toronto Convention ...,NaN
2,563d2f2dcd10bcf27ae37c3b,Pilot Coffee Roasters,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612105369,False,65 Front St W,btwn Bay St & York St,43.645018,-79.380415,"[{'label': 'display', 'lat': 43.64501814464698...",800,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W (btwn Bay St & York St), Toront...",NaN
3,4b9e7808f964a52091e636e3,Second Cup Coffee Co.,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612105369,False,200 Front St W,in Simcoe Place,43.645009,-79.385812,"[{'label': 'display', 'lat': 43.64500941272016...",461,M5V 3K2,CA,NaN,Toronto,ON,Canada,"[200 Front St W (in Simcoe Place), Toronto ON ...",NaN
4,584593c2ebf0284fe7b103cb,Fahrenheit Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612105369,False,529 Richmond St W,NaN,43.647037,-79.400876,"[{'label': 'display', 'lat': 43.64703669923361...",1114,NaN,CA,Fashion District,Toronto,ON,Canada,"[529 Richmond St W, Toronto ON, Canada]",NaN


In [24]:

filtered_columns = ['id','name', 'categories'] + [col for col in df.columns if col.startswith('location.')]
df_filtered = df.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]


df_filtered['rating'] = 0

df_filtered.head()

,id,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,rating
0,59f784dd28122f14f9d5d63d,HotBlack Coffee,Coffee Shop,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",969,M5V 1Z4,CA,Entertainment District,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",0
1,4b05a67cf964a52034e022e3,Timothy's World Coffee,Coffee Shop,225 Front Street E,Metro Toronto Convention Centre,43.640346,-79.389881,"[{'label': 'display', 'lat': 43.640346, 'lng':...",156,NaN,CA,NaN,Toronto,ON,Canada,[225 Front Street E (Metro Toronto Convention ...,0
2,563d2f2dcd10bcf27ae37c3b,Pilot Coffee Roasters,Coffee Shop,65 Front St W,btwn Bay St & York St,43.645018,-79.380415,"[{'label': 'display', 'lat': 43.64501814464698...",800,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W (btwn Bay St & York St), Toront...",0
3,4b9e7808f964a52091e636e3,Second Cup Coffee Co.,Coffee Shop,200 Front St W,in Simcoe Place,43.645009,-79.385812,"[{'label': 'display', 'lat': 43.64500941272016...",461,M5V 3K2,CA,NaN,Toronto,ON,Canada,"[200 Front St W (in Simcoe Place), Toronto ON ...",0
4,584593c2ebf0284fe7b103cb,Fahrenheit Coffee,Coffee Shop,529 Richmond St W,NaN,43.647037,-79.400876,"[{'label': 'display', 'lat': 43.64703669923361...",1114,NaN,CA,Fashion District,Toronto,ON,Canada,"[529 Richmond St W, Toronto ON, Canada]",0


In [25]:
for idx in df_filtered.index:
    venue_id = df_filtered["id"][idx]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        df_filtered.loc[idx,'rating'] = result['response']['venue']['rating']
    except:
        df_filtered.loc[idx,'rating'] = 0

In [27]:
df_best = df_filtered.sort_values('rating',ascending=False)[0:5]
df_best

,id,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,rating
10,4fff1f96e4b042ae8acddca5,Fahrenheit Coffee,Coffee Shop,120 Lombard St,at Jarvis St,43.652384,-79.372719,"[{'label': 'display', 'lat': 43.65238358726612...",1784,M5C 3H5,CA,NaN,Toronto,ON,Canada,"[120 Lombard St (at Jarvis St), Toronto ON M5C...",9.1
42,5a6b737b35f98359eed11974,The Library Specialty Coffee,Coffee Shop,281 Dundas St West,St Patrick And Dundas St W,43.654413,-79.390902,"[{'label': 'display', 'lat': 43.65441282740799...",1425,M5T 2W5,CA,NaN,Toronto,ON,Canada,[281 Dundas St West (St Patrick And Dundas St ...,8.6
16,51438b33e4b0a40e33fe5e77,Jimmy's Coffee,Café,191 Baldwin St,Kensington Market,43.654493,-79.401311,"[{'label': 'display', 'lat': 43.65449315540114...",1729,NaN,CA,Kensington Market,Toronto,ON,Canada,"[191 Baldwin St (Kensington Market), Toronto O...",8.5
39,57963897498e6873394a559d,Versus Coffee,Coffee Shop,70 Adelaide St E,at Church St,43.651213,-79.375236,"[{'label': 'display', 'lat': 43.65121343781903...",1547,NaN,CA,St. Lawrence,Toronto,ON,Canada,"[70 Adelaide St E (at Church St), Toronto ON, ...",8.4
4,584593c2ebf0284fe7b103cb,Fahrenheit Coffee,Coffee Shop,529 Richmond St W,NaN,43.647037,-79.400876,"[{'label': 'display', 'lat': 43.64703669923361...",1114,NaN,CA,Fashion District,Toronto,ON,Canada,"[529 Richmond St W, Toronto ON, Canada]",8.4


In [28]:
df_best[["name","address","postalCode","rating"]]

,name,address,postalCode,rating
10,Fahrenheit Coffee,120 Lombard St,M5C 3H5,9.1
42,The Library Specialty Coffee,281 Dundas St West,M5T 2W5,8.6
16,Jimmy's Coffee,191 Baldwin St,NaN,8.5
39,Versus Coffee,70 Adelaide St E,NaN,8.4
4,Fahrenheit Coffee,529 Richmond St W,NaN,8.4


In [32]:
map_cafe = folium.Map(location=[latitude, longitude], zoom_start=15)
label = folium.Popup(address, parse_html=True)
folium.Marker([latitude, longitude],icon=folium.Icon(color="red", prefix="fa", icon="building"),\
              popup=label).add_to(map_cafe)

for lat, lon, name, rating in zip(df_best['lat'], df_best['lng'], df_best['name'], df_best['rating']):
    label = folium.Popup(str(name) + ". " + "Rating: " + str(rating), parse_html=True)
    folium.Marker([lat, lon],icon=folium.Icon(color="blue",prefix="fa",icon="coffee"),\
                        popup=label).add_to(map_cafe)
    

map_cafe